In [3]:
import cv2
import os
import numpy as np
from cv2 import dnn_superres




In [3]:
# Create an SR object
def upscale_image(image_name,input_directory,output_directory):
    sr = dnn_superres.DnnSuperResImpl.create()
    # Read image
    image = cv2.imread(os.path.join(input_directory,image_name))
    #print(os.path.join(input_directory,image_name))
    sr.readModel('LapSRN_x2.pb')  # Update with your model file path
    sr.setModel('lapsrn', 2)  # You can choose a different scale factor

    # Upscale the image
    result = sr.upsample(image)
    #print(os.path.join(output_directory,image_name))
    cv2.imwrite(output_directory, result)


In [3]:
def upscale_images_in_directory(input_directory, output_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate through each file in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            # Construct the full path for the input and output images
            output_path = os.path.join(output_directory, filename)
            # Upscale the image and get the path to the upscaled image
            upscale_image(filename,input_directory, output_path)
            # Print the paths for verification)

# Example usage
input_directory = "training_data/data V8 buildings+planes/train/images"
output_directory = "upscaled_images"
upscale_images_in_directory(input_directory, output_directory)

In [22]:
image = cv2.imread('image_5 copy.jpeg')
sharpen_kernel = np.array([[-1,-1,-1], [-1,10,-1], [-1,-1,-1]])
sharpen = cv2.filter2D(image, -1, sharpen_kernel)
cv2.imwrite('sharpened_image.jpg', sharpen)
#cv2.imshow('sharpen', sharpen)
#cv2.waitKey(1)

True

In [5]:
def rotate_images_and_labels(input_img_dir, input_label_dir, output_img_dir, output_label_dir):
    # Create output directories if they don't exist
    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    # Iterate through each image in the input directory
    for img_filename in os.listdir(input_img_dir):
        if img_filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(input_img_dir, img_filename)

            # Read the image
            img = cv2.imread(img_path)

            # Get corresponding label file
            label_filename = img_filename.replace(os.path.splitext(img_filename)[1], '.txt')
            label_path = os.path.join(input_label_dir, label_filename)

            # Read the YOLO-style bounding box coordinates from the label file
            with open(label_path, 'r') as label_file:
                lines = label_file.readlines()

            # Create a copy of the image for rotation
            rotated_img = img.copy()

            # Rotate the image and update the bounding box coordinates
            angle = np.random.uniform(-15, 15)  # Random rotation between -15 and 15 degrees
            height, width = img.shape[:2]
            rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
            rotated_img = cv2.warpAffine(rotated_img, rotation_matrix, (width, height))

            for line in lines:
                values = line.strip().split('/n')
                print(values)
                class_id, x_center, y_center, box_width, box_height = map(float, values[1:])
                
                # Rotate bounding box coordinates
                x_center_rot = (x_center * width - width / 2) * np.cos(np.radians(angle)) - (y_center * height - height / 2) * np.sin(np.radians(angle)) + width / 2
                y_center_rot = (x_center * width - width / 2) * np.sin(np.radians(angle)) + (y_center * height - height / 2) * np.cos(np.radians(angle)) + height / 2

                # Update the rotated label
                values[1] = str(x_center_rot / width)  # Update x_center
                values[2] = str(y_center_rot / height)  # Update y_center

                rotated_label_filename = img_filename.replace(os.path.splitext(img_filename)[1], f'_rotated_{int(angle)}{os.path.splitext(img_filename)[1]}')
                rotated_img_path = os.path.join(output_img_dir, rotated_label_filename)
                rotated_label_path = os.path.join(output_label_dir, rotated_label_filename.replace(os.path.splitext(rotated_label_filename)[1], '.txt'))

                # Save the rotated image and label
                cv2.imwrite(rotated_img_path, rotated_img)
                with open(rotated_label_path, 'w') as rotated_label_file:
                    rotated_label_file.write(' '.join(map(str, values)))

# Example usage:
input_img_directory = 'test_images'
input_label_directory = 'test_labels'
output_img_directory = 'output/images'
output_label_directory = 'output/labels'

rotate_images_and_labels(input_img_directory, input_label_directory, output_img_directory, output_label_directory)

['0 0.40229885000000004 0.5111607140625 0.48491379375 0.5386904765625 0.5079022984375 0.484375 0.4295977015625 0.44940476250000005 0.40229885000000004 0.5111607140625']


ValueError: not enough values to unpack (expected 5, got 0)

In [20]:
import cv2
import os

def apply_blur_to_images(input_dir, output_dir, blur_strength):
    # Check if the input directory exists
    if not os.path.exists(input_dir):
        print(f"Error: Input directory '{input_dir}' not found.")
        return
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory '{output_dir}'.")

    # Get a list of all files in the input directory
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Apply blur to each image and save it to the output directory
    for image_file in image_files:
        image_path = os.path.join(input_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)

        # Check if the image is loaded successfully
        if image is not None:
            # Apply blur
            blurred_image = cv2.GaussianBlur(image, (blur_strength, blur_strength), 0)

            # Save the blurred image to the output directory
            output_path = os.path.join(output_dir, f"{image_file}")
            cv2.imwrite(output_path, blurred_image)
            print(f"Blur applied to '{image_file}' and saved as '{output_path}'.")
        else:
            print(f"Error loading '{image_file}'. Skipping.")

# Example usage:
input_directory = 'data V8 version 4 copy/train/images'
output_directory = 'blurred_images_v2'
blur_strength = 7  # Adjust the blur strength as needed

apply_blur_to_images(input_directory, output_directory, blur_strength)


Created output directory 'blurred_images_v2'.
Blur applied to 'mel-jalanmerdeka_png.rf.af1c742cd4a03a728a62b48b7e55b910.jpg' and saved as 'blurred_images_v2/mel-jalanmerdeka_png.rf.af1c742cd4a03a728a62b48b7e55b910.jpg'.
Blur applied to 'j-jalanmulia_png.rf.c500134212d1ace241c7d711ac6f8242.jpg' and saved as 'blurred_images_v2/j-jalanmulia_png.rf.c500134212d1ace241c7d711ac6f8242.jpg'.
Blur applied to 'per-taipinghomestayhusnaafinah_png.rf.ca47065d7c29988076cb24faa99324b3.jpg' and saved as 'blurred_images_v2/per-taipinghomestayhusnaafinah_png.rf.ca47065d7c29988076cb24faa99324b3.jpg'.
Blur applied to 'ts6_png.rf.20d6f6f535e6f2db048b226686f3af46.jpg' and saved as 'blurred_images_v2/ts6_png.rf.20d6f6f535e6f2db048b226686f3af46.jpg'.
Blur applied to 'kel-jalankoperatif_png.rf.9431ce7d95fa5b712a8a4184528f8279.jpg' and saved as 'blurred_images_v2/kel-jalankoperatif_png.rf.9431ce7d95fa5b712a8a4184528f8279.jpg'.
Blur applied to 'per-kampungbarupadangrengas_png.rf.4e0dd934e22a442acc41c26d4a4e8d9d.j

In [11]:
import cv2
import os
import numpy as np

def draw_boxes(image_size, box_coordinates, output_path):
    # Create a black image
    image = np.zeros((image_size[1], image_size[0], 3), dtype=np.uint8)

    for box in box_coordinates:
        class_label,x_center, y_center, box_width, box_height = box
        if class_label in [0, 3]:
            x_min = int((x_center - box_width / 2) * image_size[0])
            y_min = int((y_center - box_height / 2) * image_size[1])
            x_max = int((x_center + box_width / 2) * image_size[0])
            y_max = int((y_center + box_height / 2) * image_size[1])

            # Draw a filled rectangle on the image
            colour = (255,255,255)
            if class_label == 3:
                colour = (0,0,255)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), colour, -1)

    # Save or display the result
    cv2.imwrite(output_path, image)


def read_box_coordinates(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extract box coordinates from all lines
    box_coordinates = [list(map(float, line.split())) for line in lines]

    return box_coordinates

def generate_footprint(image_directory_path):
    # Directory containing TXT files
    txt_directory = "test_text"


    # Output directory for images with drawn squares
    output_directory = 'footprints'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate through each TXT file in the directory
    for txt_file in os.listdir(txt_directory):
        if txt_file.endswith(".txt"):
            # Read box coordinates from the TXT file
            txt_file_path = os.path.join(txt_directory, txt_file)
            txt_file_name = os.path.splitext(os.path.basename(txt_file_path))[0]

      # Search for an image with the same name in the image directory
        for image_file in os.listdir(image_directory_path):
          if image_file.startswith(txt_file_name) and (image_file.lower().endswith('.jpeg') or image_file.lower().endswith('.jpg')):
            # Combine the image file path and the output directory
            image_path = os.path.join(image_directory_path, image_file)

            # Read the image and save it with the desired output name
            image = cv2.imread(image_path)
            image_height, image_width, _ = image.shape
            box_coordinates = read_box_coordinates(txt_file_path)

            # Define the image size (adjust according to your needs)
            image_size = (image_width, image_height)

            # Draw filled white boxes on a black image
            output_path = os.path.join(output_directory, txt_file.replace(".txt", "_output_2.png"))
            draw_boxes(image_size, box_coordinates, output_path)

generate_footprint("test_data")

In [19]:
from PIL import Image

def split_image(image_path, threshold_width):
    # Open the image
    img = Image.open(image_path)

    # Get the width of the image
    img_width, img_height = img.size

    # Check if the image size is larger than the threshold
    if img_width > threshold_width:
        # Calculate the midpoint
        midpoint = img_width // 2

        # Crop the left half
        left_half = img.crop((0, 0, midpoint, img_height))

        # Crop the right half
        right_half = img.crop((midpoint, 0, img_width, img_height))

        # Save the two halves
        left_half.save("left_half_6.jpg")
        right_half.save("right_half_6.jpg")

        print("Image successfully split into left and right halves.")
    else:
        print("Image does not exceed the threshold width.")

# Example usage:
image_path = "test_data/image_6.jpeg"
threshold_width = 800  # Set your threshold width here
split_image(image_path, threshold_width)


Image successfully split into left and right halves.


In [ ]:
def convert_to_grayscale(input_dir, output_dir):
    # Check if the input directory exists
    if not os.path.exists(input_dir):
        print(f"Error: Input directory '{input_dir}' not found.")
        return
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory '{output_dir}'.")

    # Get a list of all files in the input directory
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Convert each image to grayscale and save it to the output directory
    for image_file in image_files:
        image_path = os.path.join(input_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)

        # Check if the image is loaded successfully
        if image is not None:
            # Convert to grayscale
            grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Save the grayscale image to the output directory
            output_path = os.path.join(output_dir, f"{image_file}")
            cv2.imwrite(output_path, grayscale_image)
            print(f"Image '{image_file}' converted to grayscale and saved as '{output_path}'.")
        else:
            print(f"Error loading '{image_file}'. Skipping.")

# Example usage:
input_directory = 'data V8 version 4 copy/train/images'
output_directory = 'output__grayscale_images_5'
convert_to_grayscale(input_directory, output_directory)

In [ ]:
def split_image_with_overlap(image_path, threshold_width, margin=10):
    # Open the image
    img = Image.open(image_path)
    img_width, img_height = img.size

    # Calculate the midpoint
    midpoint = img_width // 2

    # Calculate the new split points with margins
    split_point_left = max(0, midpoint - margin)
    split_point_right = min(img_width, midpoint + margin)

    # Crop the left half with margin
    left_half_img = img.crop((0, 0, split_point_left, img_height))
    left_half_img.save("left_half.jpg")

    # Crop the right half with margin
    right_half_img = img.crop((split_point_right, 0, img_width, img_height))
    right_half_img.save("right_half.jpg")

    print("Image successfully split into left and right halves with overlap.")

# Example usage:
image_path = "path/to/your/image.jpg"
threshold_width = 500  # Set your threshold width here
margin = 10  # Set your desired margin here
split_image_with_overlap(image_path, threshold_width, margin)

In [4]:
image = cv2.imread('image_4 copy.jpeg')

# define the alpha and beta
alpha = 1.5 # Contrast control
beta = 10 # Brightness control

# call convertScaleAbs function
adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

cv2.imwrite('bright_4.jpeg', adjusted)

True

In [9]:
from PIL import Image
import cv2 as cv

def zoom_at(img, x, y, zoom):
    w, h = img.size
    zoom2 = zoom * 2
    img = img.crop((x - w / zoom2, y - h / zoom2, 
                    x + w / zoom2, y + h / zoom2))
    return img.resize((w, h), Image.LANCZOS)

img = Image.open("image_4 copy.jpeg")

img = zoom_at(img, 264.5, 275, 1.5)

img = img.save('image_4 zoomed.jpeg')

In [6]:
import os

def replace_class_in_txt(directory, new_class):
    # Iterate through each TXT file in the directory
    for txt_file in os.listdir(directory):
        if txt_file.endswith(".txt"):
            # Read lines from the TXT file
            file_path = os.path.join(directory, txt_file)
            with open(file_path, 'r') as file:
                lines = file.read().split('\n')

            # Replace the class in each line with the new class
            updated_lines = [f"{new_class} {' '.join(line.split()[1:])}" for line in lines]

            # Join the updated lines with newline character
            updated_content = '\n'.join(updated_lines)

            # Write the updated content back to the TXT file
            with open(file_path, 'w') as file:
                file.write(updated_content)

# Example usage:
txt_directory = r'C:\Users\Max\Desktop/train/labels'
new_class = 3  # Replace with the desired new class
replace_class_in_txt(txt_directory, new_class)


In [4]:
import json

def yolo_to_geojson(yolo_filepath, geojson_filepath):
    with open(yolo_filepath, 'r') as yolo_file:
        yolo_data = yolo_file.readlines()

    features = []

    for line in yolo_data:
        # Extracting x_center, y_center, box_width, box_height from YOLO format
        x_center, y_center, box_width, box_height = map(float, line[1:].strip().split(' '))

        # Converting YOLO coordinates to GeoJSON coordinates
        min_x = x_center - box_width / 2
        min_y = y_center - box_height / 2
        max_x = x_center + box_width / 2
        max_y = y_center + box_height / 2

        # Creating GeoJSON feature
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [
                    [
                        [min_x, min_y],
                        [min_x, max_y],
                        [max_x, max_y],
                        [max_x, min_y]
                    ]
                ]
            },
            'properties': {
                'class': 'building'  # You can modify this to include additional properties
            }
        }

        features.append(feature)

    # Creating GeoJSON feature collection
    geojson_data = {
        'type': 'FeatureCollection',
        'features': features
    }

    # Writing to GeoJSON file
    with open(geojson_filepath, 'w') as geojson_file:
        json.dump(geojson_data, geojson_file, indent=2)

# Example usage:
yolo_to_geojson(r'Rubble Detection.v6i.yolov8\test\labels\5_PNG.rf.dc97d42dec4946d48c726f69d73f1ea7.txt', 'test')


In [23]:
import json

def yolo_to_geojson(yolo_filepath, geojson_filepath):
        with open(yolo_filepath, 'r') as yolo_file:
            yolo_data = yolo_file.readlines()

        features = []

        for line in yolo_data:
            # Extracting class, x_center, y_center, box_width, box_height from YOLO format
            class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split(' '))

            # Check if the class is of interest (class 0 or class 3)
            if class_id in [0, 3]:
                # Converting YOLO coordinates to GeoJSON coordinates
                min_x = x_center - box_width / 2
                min_y = y_center - box_height / 2
                max_x = x_center + box_width / 2
                max_y = y_center + box_height / 2

                # Set damage property based on class
                damage_property = 'none' if class_id == 0 else 'destroyed'
                object = 'building' if class_id == 0 else 'rubble'
                # Creating GeoJSON feature
                feature = {
                    'type': 'Feature',
                    'geometry': {
                        'type': 'Polygon',
                        'coordinates': [
                            [min_x, min_y],
                            [max_x, min_y],
                            [max_x, max_y],
                            [min_x, max_y]
                        ]
                    },
                    'properties': {
                        'class': object,
                        'damage': damage_property
                    }
                }

                features.append(feature)

        # Creating GeoJSON feature collection
        geojson_data = {
            'type': 'FeatureCollection',
            'features': features
        }

        # Writing to GeoJSON file
        with open(geojson_filepath, 'w') as geojson_file:
            json.dump(geojson_data, geojson_file, indent=2)

        print(f"Conversion completed. GeoJSON file saved to {geojson_filepath}")


# Example usage:
yolo_to_geojson('test_text/image_5.txt', 'test_data_1')


Conversion completed. GeoJSON file saved to test_data_1


In [12]:
def overlay_images(image_path1, image_path2,output_path, alpha=0.7):
    # Load the two images
    image1 = cv2.imread(image_path1)
    image2 = cv2.imread(image_path2)

    # Make sure both images have the same dimensions
    image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]))

    # Specify the weight for each image in the overlay (alpha blending)
    beta = 1 - alpha

    # Perform the overlay by blending the two images
    overlay = cv2.addWeighted(image1, alpha, image2, beta, 0)

    # Display or save the result
    cv2.imwrite(output_path, overlay)

# Example usage:
image_path1 = 'test_data\image_5.jpeg'
image_path2 = 'footprints\image_5_output_2.png'
output_path = 'overlaid.jpg'
overlay_images(image_path1, image_path2, output_path)

In [28]:
import os
import cv2
import numpy as np

def generate_footprint(image_directory_path):
    def draw_boxes(image_size, box_coordinates, output_path):
        image = np.zeros((image_size[1], image_size[0], 3), dtype=np.uint8)

        for box in box_coordinates:
            class_label, x_center, y_center, box_width, box_height = box
            if class_label in [0, 3]:
                x_min = int((x_center - box_width / 2) * image_size[0])
                y_min = int((y_center - box_height / 2) * image_size[1])
                x_max = int((x_center + box_width / 2) * image_size[0])
                y_max = int((y_center + box_height / 2) * image_size[1])

                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 255, 255), -1)

        cv2.imwrite(output_path, image)

    def read_box_coordinates(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()

        box_coordinates = [list(map(float, line.split())) for line in lines]

        return box_coordinates

    txt_directory = "test_text"
    output_directory = 'footprints'
    overlay_directory = 'overlays'  # New directory for overlaid images
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    if not os.path.exists(overlay_directory):
        os.makedirs(overlay_directory)

    for txt_file in os.listdir(txt_directory):
        if txt_file.endswith(".txt"):
            txt_file_path = os.path.join(txt_directory, txt_file)
            txt_file_name = os.path.splitext(os.path.basename(txt_file_path))[0]

            for image_file in os.listdir(image_directory_path):
                if image_file.startswith(txt_file_name) and (image_file.lower().endswith('.jpeg') or image_file.lower().endswith('.jpg')):
                    image_path = os.path.join(image_directory_path, image_file)
                    image = cv2.imread(image_path)
                    image_height, image_width, _ = image.shape
                    box_coordinates = read_box_coordinates(txt_file_path)
                    image_size = (image_width, image_height)

                    # Draw filled white boxes on a black image
                    output_path = os.path.join(output_directory, txt_file.replace(".txt", "_footprint.png"))
                    draw_boxes(image_size, box_coordinates, output_path)
                    footprint_path = os.path.join(overlay_directory, txt_file.replace(".txt", "_overlay.png"))
                    # Overlay the footprint on the original image
                    overlay_path = os.path.join(output_path, txt_file.replace(".txt", "_footprint.png"))
                    overlay_images(image_path, footprint_path,output_path, overlay_path)

# Example usage:
generate_footprint("YOLO_predictions/Latest_run")


TypeError: unsupported operand type(s) for -: 'int' and 'str'